# Data Source

In [1]:
data_path = "/Users/alexander.girardet/Code/Personal/projects/rag_poc/data/Articles.csv"

In [2]:
import pandas as pd

In [5]:
articles = pd.read_csv(data_path , encoding='ISO-8859-1')

In [12]:
articles.head()

,Article,Date,Heading,NewsType
0,KARACHI: The Sindh government has decided to b...,1/1/2015,sindh govt decides to cut public transport far...,business
1,HONG KONG: Asian markets started 2015 on an up...,1/2/2015,asia stocks up in new year trad,business
2,HONG KONG: Hong Kong shares opened 0.66 perce...,1/5/2015,hong kong stocks open 0.66 percent lower,business
3,HONG KONG: Asian markets tumbled Tuesday follo...,1/6/2015,asian stocks sink euro near nine year,business
4,NEW YORK: US oil prices Monday slipped below $...,1/6/2015,us oil prices slip below 50 a barr,business


In [20]:
articles['Date'] = pd.to_datetime(articles['Date']) # In flight data processing during extraction.

In [18]:
articles.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2692 entries, 0 to 2691
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   Article   2692 non-null   object        
 1   Date      2692 non-null   datetime64[ns]
 2   Heading   2692 non-null   object        
 3   NewsType  2692 non-null   object        
dtypes: datetime64[ns](1), object(3)
memory usage: 84.3+ KB


# Data Extraction process

Extract all the data to a standardized JSON format for MongoDB storage. Acting as the unstructured data store.

In [ ]:
from pymongo import MongoClient

client = MongoClient("mongodb://localhost:27017/") # Hosted with Docker

db = client["unstructured_data_store"]

# Access collection
collection = db["articles"]

## Article Data model

In [28]:
from pydantic import BaseModel, Field
from typing import Optional
from datetime import datetime
from tqdm import tqdm

In [21]:
class NewsArticle(BaseModel):
    Article: str
    Heading: str
    Date: datetime
    NewsType: str

In [24]:
article_batches = articles.to_dict(orient="records")

In [30]:
for article_json in tqdm(article_batches):
    article = NewsArticle(**article_json)
    article_processed_json = article.dict()
    collection.insert_one(article_processed_json)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 2692/2692 [00:01<00:00, 1427.13it/s]


All articles are loaded into MongoDB collection. 